<h2> How to work with Amazon Bedrock Guardrails</h2>

*Note: This notebook has been adapted from the [Guardrails for Amazon Bedrock - Examples using the Python SDK](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/responsible-ai/guardrails-for-amazon-bedrock-samples/guardrails-api.ipynb)*

<h2> Overview </h2>

This notebook demonstrates using Amazon Bedrock Guardrails by creating, updating and testing the Guardrail Policy config using Amazon Bedrock APIs. Guardrail evaluates user inputs and FM responses based on use case specific policies, and provides an additional layer of safeguards regardless of the underlying FM. Guardrails can be applied across all large language models (LLMs) on Amazon Bedrock, including fine-tuned models. Customers can create multiple guardrails, each configured with a different combination of controls, and use these guardrails across different applications and use cases. 

<h2> Context </h2>

Guardrails can be used to implement safeguards for your generative AI applications that are customized to your use cases and aligned with your responsible AI policies. Guardrails allows you to configure the following policies in a guardrail to avoid undesirable and harmful content and remove sensitive information for privacy protection.

- **Content filters** – Adjust filter strengths to block input prompts or model responses containing harmful content.
- **Denied topics** – Define a set of topics that are undesirable in the context of your application. These topics will be blocked if detected in user queries or model responses.
- **Word filters** – Configure filters to block undesirable words, phrases, and profanity. Such words can include offensive terms, competitor names etc.
- **Sensitive information filters** – Block or mask sensitive information such as personally identifiable information (PII) or custom regex in user inputs and model responses.
- **Contextual grounding check** – Detect and filter hallucinations in model responses based on grounding in a source and relevance to the user query.

For more information on publicly available capabilities:

- [Documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails.html)
- [Guardrail Policies](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-components.html)
- [Pricing](https://aws.amazon.com/bedrock/pricing/)
- [WebPage](https://aws.amazon.com/bedrock/guardrails/)

<h3> Architecture </h3>

![How Guardrails for Amazon Bedrock Works](assets/how-to-guardrails-architecture.png)

<h2> Prerequisites </h2>

- Amazon Bedrock basic setup has been completed, see `Prerequisites` section under [Amazon Bedrock APIs - Getting Started](01_invoke_api.ipynb)
- Amazon Bedrock access to below given Foundation Model used in this notebook, in `us-east-1` (N. Virginia) region.

| Provider Name | Foundation Model Name | Model Id |
| ------- | ------------- | ------------- |
| Anthropic | Claude 3 Haiku  | anthropic.claude-3-haiku-20240307-v1:0 |


<h2> Setup </h2>

⚠️ This notebook should work well with the Data Science 3.0 kernel (Python 3.10 runtime) in SageMaker Studio ⚠️

In [4]:
#Start by installing the dependencies to ensure we have a recent version
!pip3 install --upgrade --force-reinstall boto3
import boto3
print(boto3.__version__)

Defaulting to user installation because normal site-packages is not writeable
  Using cached boto3-1.37.3-py3-none-any.whl (139 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.11.3-py3-none-any.whl (84 kB)
  Using cached botocore-1.37.3-py3-none-any.whl (13.3 MB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
  Using cached urllib3-1.26.20-py2.py3-none-any.whl (144 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.20
    Uninstalling urllib3-1.26.20:
      Successfully uninstalled urllib3-1.26.20
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.9.0.post0
    Uninstalling python-dateutil-2.9.0.post0:
      Successfully uninstalled python-dateutil-2.9.0.pos

<h2>Notebook/Code with comments</h2>

<h3> Creating a Guardrail </h3>

We will use `create_guardrail` API to create an example Guardrail `fiduciary-advice` by setting up the configurations, defining topics to deny, providing filters to handle harmful and sensitive content, and writing messages for when prompts and user responses are blocked. 

In [6]:
"""Helper utilities for working with Amazon Bedrock from Python notebooks"""
# Python Built-Ins:
import os
from typing import Optional
import sys
import json

# External Dependencies:
import boto3
from botocore.config import Config
import botocore

def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-2").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    runtime :
        Optional choice of getting different client to perform operations with the Amazon Bedrock service.
    """
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

In [7]:
module_path = ".."
sys.path.append(os.path.abspath(module_path))


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
os.environ["AWS_PROFILE"] = "yulthebad01-bedrock"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False
)

Create new client
  Using region: us-east-1
  Using profile: yulthebad01-bedrock
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


In [8]:
boto3_bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'c7a86e30-1f09-4705-8ac1-947580f53188',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 28 Feb 2025 15:37:49 GMT',
   'content-type': 'application/json',
   'content-length': '41646',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c7a86e30-1f09-4705-8ac1-947580f53188'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

In [9]:
region = 'us-east-1'
print('Using region: ', region)

# client = boto3.client(
#     service_name = 'bedrock',
#     region_name = region,
#     )

create_response = boto3_bedrock.create_guardrail(
    name='fiduciary-advice',
    description='Prevents the our model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.75
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.75
            }
        ]
    },
    blockedInputMessaging="""I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    blockedOutputsMessaging="""I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    tags=[
        {'key': 'purpose', 'value': 'fiduciary-advice-prevention'},
        {'key': 'environment', 'value': 'production'}
    ]
)

print(create_response)

Using region:  us-east-1
{'ResponseMetadata': {'RequestId': '3f49435b-a3ae-4e8a-aa5b-9c93b7a12e38', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Fri, 28 Feb 2025 15:38:37 GMT', 'content-type': 'application/json', 'content-length': '172', 'connection': 'keep-alive', 'x-amzn-requestid': '3f49435b-a3ae-4e8a-aa5b-9c93b7a12e38'}, 'RetryAttempts': 0}, 'guardrailId': 'pn0zblegjuyl', 'guardrailArn': 'arn:aws:bedrock:us-east-1:535002878551:guardrail/pn0zblegjuyl', 'version': 'DRAFT', 'createdAt': datetime.datetime(2025, 2, 28, 15, 38, 37, 510762, tzinfo=tzutc())}


<h3> Getting a Guardrail, creating a version and listing all the versions and Drafts </h3>

- The `working draft` is a version of the guardrail that you can continually edit and iterate upon until you reach a satisfactory configuration for your use case. 
- A version of the guardrail acts as a snapshot of the configurations of the `working draft` when you create the version. You can use versions to streamline guardrails deployment to production applications every time you make modifications to your guardrails.

In [10]:
#This will provide all the data about the DRAFT version we have

get_response = boto3_bedrock.get_guardrail(
    guardrailIdentifier=create_response['guardrailId'],
    guardrailVersion='DRAFT'
)


In [11]:
# Now let's create a version for our Guardrail 

version_response = boto3_bedrock.create_guardrail_version(
    guardrailIdentifier=create_response['guardrailId'],
    description='Version of Guardrail'
)

In [12]:
# To list the DRAFT version of all your guardrails, don’t specify the guardrailIdentifier field. To list all versions of a guardrail, specify the ARN of the guardrail in the guardrailIdentifier field.

list_guardrails_response = boto3_bedrock.list_guardrails(
    guardrailIdentifier=create_response['guardrailArn'],
    maxResults=5)

print(list_guardrails_response)

{'ResponseMetadata': {'RequestId': '09a70823-b3e9-43a9-bd49-679eef4de35c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 28 Feb 2025 15:40:38 GMT', 'content-type': 'application/json', 'content-length': '604', 'connection': 'keep-alive', 'x-amzn-requestid': '09a70823-b3e9-43a9-bd49-679eef4de35c'}, 'RetryAttempts': 0}, 'guardrails': [{'id': 'pn0zblegjuyl', 'arn': 'arn:aws:bedrock:us-east-1:535002878551:guardrail/pn0zblegjuyl', 'status': 'READY', 'name': 'fiduciary-advice', 'description': 'Prevents the our model from providing fiduciary advice.', 'version': 'DRAFT', 'createdAt': datetime.datetime(2025, 2, 28, 15, 38, 37, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2025, 2, 28, 15, 40, 26, 642240, tzinfo=tzutc())}, {'id': 'pn0zblegjuyl', 'arn': 'arn:aws:bedrock:us-east-1:535002878551:guardrail/pn0zblegjuyl', 'status': 'READY', 'name': 'fiduciary-advice', 'description': 'Version of Guardrail', 'version': '1', 'createdAt': datetime.datetime(2025, 2, 28, 15, 40, 26, tzinfo=tzutc())

<h3> Updating a Guardrail  </h3>


Let's update the Guardrail using `update_guardrail` API by modifing one of our content filters (`HATE` to `MEDIUM`).

In [13]:
# Updating the Guardrail by providing another adjusting our Content Filter strength 

response = boto3_bedrock.update_guardrail(
    guardrailIdentifier=create_response['guardrailArn'],
    name='fiduciary-advice',
    description='Prevents the our model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'MEDIUM',
                'outputStrength': 'MEDIUM'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.75
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.75
            }
        ]
    },
    blockedInputMessaging="""I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    blockedOutputsMessaging="""I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
)



In [14]:
# Let's now get all of our updates 
get_response = boto3_bedrock.get_guardrail(
    guardrailIdentifier=create_response['guardrailId'],
    guardrailVersion='DRAFT'
)
print(get_response)

{'ResponseMetadata': {'RequestId': 'ab4a89d7-38d2-4853-82d3-5aa6a21c09f6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 28 Feb 2025 15:41:42 GMT', 'content-type': 'application/json', 'content-length': '5568', 'connection': 'keep-alive', 'x-amzn-requestid': 'ab4a89d7-38d2-4853-82d3-5aa6a21c09f6'}, 'RetryAttempts': 0}, 'name': 'fiduciary-advice', 'description': 'Prevents the our model from providing fiduciary advice.', 'guardrailId': 'pn0zblegjuyl', 'guardrailArn': 'arn:aws:bedrock:us-east-1:535002878551:guardrail/pn0zblegjuyl', 'version': 'DRAFT', 'status': 'READY', 'topicPolicy': {'topics': [{'name': 'Fiduciary Advice', 'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.', 'examples': ['What stocks should I invest in for my retirement?', 'Is it a good idea to put my money in a mutual fund?', 'How should I allocate my 401(k) investments?', 'Wh

In [15]:
# Create a new Version from our updates 
version_response = boto3_bedrock.create_guardrail_version(
    guardrailIdentifier=create_response['guardrailId'],
    description='Version of Guardrail that has a MEDIUM Hate Filter'
)

In [16]:
# Get all of our Guardrails 
list_guardrails_response = boto3_bedrock.list_guardrails(
    guardrailIdentifier=create_response['guardrailArn'],
    maxResults=5)

print(list_guardrails_response)

{'ResponseMetadata': {'RequestId': '0951ee03-3cec-4b92-b7c3-97f1d0f77543', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 28 Feb 2025 15:42:08 GMT', 'content-type': 'application/json', 'content-length': '900', 'connection': 'keep-alive', 'x-amzn-requestid': '0951ee03-3cec-4b92-b7c3-97f1d0f77543'}, 'RetryAttempts': 0}, 'guardrails': [{'id': 'pn0zblegjuyl', 'arn': 'arn:aws:bedrock:us-east-1:535002878551:guardrail/pn0zblegjuyl', 'status': 'READY', 'name': 'fiduciary-advice', 'description': 'Prevents the our model from providing fiduciary advice.', 'version': 'DRAFT', 'createdAt': datetime.datetime(2025, 2, 28, 15, 38, 37, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2025, 2, 28, 15, 41, 56, 782522, tzinfo=tzutc())}, {'id': 'pn0zblegjuyl', 'arn': 'arn:aws:bedrock:us-east-1:535002878551:guardrail/pn0zblegjuyl', 'status': 'READY', 'name': 'fiduciary-advice', 'description': 'Version of Guardrail', 'version': '1', 'createdAt': datetime.datetime(2025, 2, 28, 15, 40, 26, tzinfo=tzutc())

<h3> Testing our Guardrail  </h3>

We will test our Guardrails with Amazon Bedrock using `Anthropic Claude 3 Haiku` LLM Model and latest version of `fiduciary-advice` Guardrail configuration.

In [17]:
#import the run-time client
import json
# bedrock_runtime = boto3.client('bedrock-runtime', region_name = region)

In [19]:
bedrock_runtime = get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

def invoke_model(body, model_id, accept, content_type):
    """
    Invokes Amazon bedrock model to run an inference
    using the input provided in the request body.
    
    Args:
        body (dict): The invokation body to send to bedrock
        model_id (str): the model to query
        accept (str): input accept type
        content_type (str): content type
    Returns:
        Inference response from the model.
    """

    try:
        response = bedrock_runtime.invoke_model(
            body=json.dumps(body), 
            modelId=model_id, 
            accept=accept, 
            contentType=content_type
        )

        return response

    except Exception as e:
        print(f"Couldn't invoke {model_id}")
        raise e

Create new client
  Using region: us-east-1
  Using profile: yulthebad01-bedrock
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [21]:
# Build our request to Bedrock, we will test our second version

payload = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "How should I invest for my retirement? I want to be able to generate $5,000 a month"
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
response = bedrock_runtime.invoke_model(
    body = body_bytes,
    contentType = payload['contentType'],
    accept = payload['accept'],
    modelId = payload['modelId'],
    guardrailIdentifier = create_response['guardrailId'], 
    guardrailVersion ="2", 
    trace = "ENABLED"
)

# Print the response
response_body = response['body'].read().decode('utf-8')
print(json.dumps(json.loads(response_body), indent=2))

{
  "type": "message",
  "role": "assistant",
  "content": [
    {
      "type": "text",
      "text": "I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. "
    }
  ],
  "amazon-bedrock-trace": {
    "guardrail": {
      "input": {
        "pn0zblegjuyl": {
          "topicPolicy": {
            "topics": [
              {
                "name": "Fiduciary Advice",
                "type": "DENY",
                "action": "BLOCKED"
              }
            ]
          },
          "invocationMetrics": {
            "guardrailProcessingLatency": 227,
            "usage": {
              "topicPolicyUnits": 1,
              "contentPolicyUnits": 1,
       

<h2> Next steps </h2>

Now that we have seen how to use Amazon Bedrock Guardrails, you can learn
- How to use [Amazon Bedrock Knowledge Bases](03_knowledgebases_api.ipynb)
- How to use [Amazon Bedrock Agents](04_agents_api.ipynb)
- To further explore the capabilities of Amazon Bedrock Guardrails, refer [Responsible AI and Guardrails](../../responsible_ai/).

<h2> Clean up </h2>

The next optional step is to delete Guardrail created in this notebook.

In [ ]:
guardrail_del_response = client.delete_guardrail(
    guardrailIdentifier=create_response['guardrailId']
)